In [1]:
import pandas as pd
import urllib
from lxml import etree, html

In [2]:
def get_submission_trtables(weburl):
    with urllib.request.urlopen(weburl) as response:
       page = response.read()
    doc = html.fromstring(page)
    table = doc.xpath("//*[contains(@class, 'border_white')]")[0]
    tr_elements = table.xpath('tr')
    return tr_elements

In [3]:
def get_array_from_trtable(tr_elements, filename=None):
    col=[]
    i=0

    # Make header
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        col.append((name,[]))

    for j in range(1,len(tr_elements)):
        # T is our j'th row
        T=tr_elements[j]

        # index of the column
        i=0

        # Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            if i==0:
                data= str(data).strip()
            col[i][1].append(data)
            i+=1
            
    Dict = {title:column for (title,column) in col}
    submissions = pd.DataFrame(Dict)
    if filename is not None:
        submissions.to_csv(filename + '.csv', sep='\t')
    return submissions

In [4]:
weburl = 'http://www.tractometer.org/ismrm_2015_challenge/results'
with urllib.request.urlopen(weburl) as response:
   page = response.read()
doc = html.fromstring(page)
table = doc.xpath("//*[contains(@id, 'results_table')]")[0]
tr_elements = table.xpath('//tr')
submissions = get_array_from_trtable(tr_elements, filename='scraped_submissions')

In [ ]:
submission_url = 'http://www.tractometer.org/ismrm_2015_challenge/results/{0}'
submission_tables = {}
for submission in submissions['Submission ID']:
    trelements = get_submission_trtables(submission_url.format(submission))
    submission_tables[submission] = get_array_from_trtable(trelements, filename='submission{0}'.format(submission))
    

In [ ]:
submissions

In [ ]:
submission_tables['1_0']

In [ ]:
main_table['mean overlap'] = -1
main_table['mean overreach'] = -1
for team, submission in submission_tables.items():
    ol_mean = submission['Overlap (% of GT)'].str.replace("%", "").astype(float).mean()
    or_mean = submission['Overreach (% of GT)'].str.replace("%", "").astype(float).mean()
    
    idx = main_table.index[main_table['Submission ID'] == team]
    main_table.loc[idx, 'mean overlap'] = ol_mean
    main_table.loc[idx, 'mean overreach'] = or_mean

In [ ]:
main_table